In [27]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
os.chdir("D:\\GoogleDrive\\InterviewAssignment\\GoJekDriverAcceptanceGit\\GoJekAssignment")

df_train  = pd.read_csv("data\\processed\\transformed_dataset_train.csv")
df_test  = pd.read_csv("data\\processed\\transformed_dataset_test.csv")


features=["driver_gps_accuracy",
    "trip_distance",
    "hourofday",
    "hourofday_average",
    "driver_hourofday_average",
    "dayofweek",
    "dayofweek_average",
    "driver_dayofweek_average",
    "is_accepted_last5day_rolling_mean",
    "driver_cluster_label",
    "customer_cluster_label"
    ]

#Taking single value bcs of computational resource issue 
param_grid = { 
    'n_estimators': [200,300,400],
    'max_features': ['auto'],
    'max_depth' : [10,20,30],
    'class_weight':['balanced']
}

param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,8,16,32],
    'criterion' :['entropy','gini'],
}



rf_estimator = RandomForestClassifier(n_jobs=-1,random_state=33,bootstrap=False,class_weight="balanced")
CV_rfc = GridSearchCV(estimator=rf_estimator, param_grid=param_grid, cv= 4)
CV_rfc.fit(df_train[features],df_train['is_accepted'])

GridSearchCV(cv=4,
             estimator=RandomForestClassifier(bootstrap=False,
                                              class_weight='balanced',
                                              n_jobs=-1, random_state=33),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': [4, 8, 16, 32],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [200, 500]})

In [28]:
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 32,
 'max_features': 'auto',
 'n_estimators': 200}

In [29]:
dict(zip(features, CV_rfc.best_estimator_.feature_importances_))

{'driver_gps_accuracy': 0.013674540077657288,
 'trip_distance': 0.03159488963622775,
 'hourofday': 0.006082171021156395,
 'hourofday_average': 0.006934781106140472,
 'driver_hourofday_average': 0.4619503659641661,
 'dayofweek': 0.003940748111814644,
 'dayofweek_average': 0.0037843933139308666,
 'driver_dayofweek_average': 0.34747332811275744,
 'is_accepted_last5day_rolling_mean': 0.1119503992572772,
 'driver_cluster_label': 0.006251690862789241,
 'customer_cluster_label': 0.006362692536082706}

In [32]:
from sklearn.metrics import f1_score
y_true=df_test["is_accepted"].values
y_pred=CV_rfc.best_estimator_.predict(df_test[features].values)
# y_pred=rf_estimator.predict(df_test[features].values)
metrics = {}
metrics["f_score_class_1"]=f1_score(y_true,y_pred,pos_label=1)
metrics["f_score_class_0"]=f1_score(y_true,y_pred,pos_label=0)
metrics

{'f_score_class_1': 0.9338222264629309, 'f_score_class_0': 0.2408131352619234}

In [33]:
import numpy as np 
np.unique(y_true, return_counts=True)

(array([0, 1], dtype=int64), array([ 3846, 36033], dtype=int64))